<a href="https://colab.research.google.com/github/bskang8/CVPR2023_Project/blob/main/train_cvpr_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.1 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloomz-3b",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")

In [ ]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=2560, out_features=7680, bias=True)
          (dense): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elemen

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4915200 || all params: 3007472640 || trainable%: 0.1634329082375293


In [ ]:
import transformers
from datasets import load_dataset

dataset_name = "bskang/CVPR2023_title_abstract_intro"
title= "title"
abstract = "abstract"
introduction = "introduction"

In [ ]:
dataset = load_dataset(dataset_name)
print(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract', 'introduction'],
        num_rows: 2335
    })
})


In [ ]:
def generate_prompt(title: str, abstract: str, introduction: str) -> str:
  prompt = f"### INSTRUCTION\nBelow is a paper's title and abstract, please write a introduction for this paper.\n\n### Title:\n{title}\n### Abstract:\n{abstract}\n\n### Introduction:\n{introduction}"
  return prompt

mapped_dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['title'], samples['abstract'], samples['introduction'])))

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
mapped_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract', 'introduction', 'input_ids', 'attention_mask'],
        num_rows: 300
    })
})

In [ ]:
print(generate_prompt(dataset['train'][0]['title'], dataset['train'][0]['abstract'], dataset['train'][0]['introduction']))

### INSTRUCTION
Below is a paper's title and abstract, please write a introduction for this paper.

### Title:
Dong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023
### Abstract:
Abstract Indoor scene reconstruction from monocular images has long been sought after by augmented reality and robotics developers. Recent advances in neural field representa-tions and monocular priors have led to remarkable re-sults in scene-level surface reconstructions. The reliance on Multilayer Perceptrons (MLP), however, significantly limits speed in training and rendering. In this work, we propose to directly use signed distance function (SDF) in sparse voxel block grids for fast and accurate scene recon-struction without MLPs. Our globally sparse and locally dense data structure exploits surfaces’ spatial sparsity, en-ables cache-friendly queries, and allows direct extensions to multi-modal data such as color and semantic labels. To apply this representation to monocul

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=1200,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,2.706100
2,3.341300
3,2.893600
4,3.165500
5,2.855800
6,2.771500
7,3.168600
8,3.595100
9,2.821600
10,2.611200


/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


TrainOutput(global_step=1200, training_loss=1.3619077848518888, metrics={'train_runtime': 8367.7552, 'train_samples_per_second': 0.574, 'train_steps_per_second': 0.143, 'total_flos': 1.181330809110528e+17, 'train_loss': 1.3619077848518888, 'epoch': 16.0})

In [ ]:
HUGGING_FACE_USER_NAME = "bskang"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "trained_cvpr2023_data_300"

model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:832: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bskang/trained_cvpr2023_data_300/commit/9c9f88c500a39654cbe6f5f73f0aaaf143f88b05', commit_message='Upload model', commit_description='', oid='9c9f88c500a39654cbe6f5f73f0aaaf143f88b05', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from IPython.display import display, Markdown

def make_inference(title, abstract):
  batch = tokenizer(f"### INSTRUCTION\nBelow is a paper's title and abstract, please write a introduction for this paper.\n\n### Title:\n{title}\n### Abstract:\n{abstract}\n\n### Introduction:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=2000)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
your_product_name_here = dataset['train'][0]['title']
your_product_description_here = dataset['train'][0]['abstract']

make_inference(your_product_name_here, your_product_description_here)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1494: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have re

### INSTRUCTION
Below is a paper's title and abstract, please write a introduction for this paper.

### Title:
Dong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023
### Abstract:
Abstract Indoor scene reconstruction from monocular images has long been sought after by augmented reality and robotics developers. Recent advances in neural field representa-tions and monocular priors have led to remarkable re-sults in scene-level surface reconstructions. The reliance on Multilayer Perceptrons (MLP), however, significantly limits speed in training and rendering. In this work, we propose to directly use signed distance function (SDF) in sparse voxel block grids for fast and accurate scene recon-struction without MLPs. Our globally sparse and locally dense data structure exploits surfaces’ spatial sparsity, en-ables cache-friendly queries, and allows direct extensions to multi-modal data such as color and semantic labels. To apply this representation to monocular scene reconstruc-tion, we develop a scale calibration algorithm for fast geo-metric initialization from monocular depth priors. We apply differentiable volume rendering from this initialization to refine details with fast convergence. We also introduce effi-cient high-dimensional Continuous Random Fields (CRFs) to further exploit the semantic-geometry consistency be-tween scene objects. Experiments show that our approach is 10× faster in training and 100× faster in rendering while achieving comparable accuracy to state-of-the-art neural implicit methods. 

### Introduction:
1. Introduction Reconstructing indoor scenes from monocular in-door images is a fundamental problem in computer vision and graphics with applications in augmented reality (AR), robotics, and visual cognition. Recent neural field representations [3, 4, 21, 30] and monocular priors [8, 9, 11, 12, 16, 26] have led to significant progress in scene-level surface reconstruc-tion. With the success of neural radiance repre-sentation, the recent surge of monocular scene reconstruction has been driven by the demand for fast and accurate scene recon-struction in visual cognition. While the recent neural radiance models are fast and accurate, the reliance on Multilayer Perceptrons (MLPs) limits their applicability in practice. Monocular scene reconstruction from small labeled samples remains an open problem. The recent work Sparse NeRF [21] applies a MLP to sparse grid voxel blocks to recon-struct the scene without training from large labeled

In [ ]:
dataset['train'][2000]['introduction']

'1. Introduction\nGenerating realistic and editable 3D content is a long-\nstanding problem in computer vision and graphics that has\nrecently gained more attention due to the increased demand\nfor 3D objects in AR/VR, robotics and gaming applications.\nThis CVPR paper is the Open Access version, provided by the Computer Vision Foundation.\nExcept for this watermark, it is identical to the accepted version;\nthe final published version of the proceedings is available on IEEE Xplore.\n4466\nHowever, manual creation of 3D models is a laborious en-\ndeavor that requires technical skills from highly experi-\nenced artists and product designers. On the other hand, edit-\ning 3D shapes, typically involves re-purposing existing 3D\nmodels, by manually changing faces and vertices of a mesh\nand modifying its respective UV-map [95]. To accommo-\ndate this process, several recent works introduced genera-\ntive models that go beyond generation and allow editing the\ngenerated instances [13,18,52,

In [ ]:
your_product_name_here = dataset['train'][2000]['title']
your_product_description_here = dataset['train'][2000]['abstract']

make_inference(your_product_name_here, your_product_description_here)

### INSTRUCTION
Below is a paper's title and abstract, please write a introduction for this paper.

### Title:
Tertikas_Generating_Part-Aware_Editable_3D_Shapes_Without_3D_Supervision_CVPR_2023
### Abstract:
Abstract
Impressive progress in generative models and implicit
representations gave rise to methods that can generate 3D
shapes of high quality. However, being able to locally con-
trol and edit shapes is another essential property that can
unlock several content creation applications. Local control
can be achieved with part-aware models, but existing meth-
ods require 3D supervision and cannot produce textures. In
this work, we devise PartNeRF , a novel part-aware gener-
ative model for editable 3D shape synthesis that does not
require any explicit 3D supervision. Our model generates
objects as a set of locally defined NeRFs, augmented with
an affine transformation. This enables several editing op-
erations such as applying transformations on parts, mixing
*Work done during internship at Stanford.parts from different objects etc. To ensure distinct, manip-
ulable parts we enforce a hard assignment of rays to parts
that makes sure that the color of each ray is only determined
by a single NeRF . As a result, altering one part does not af-
fect the appearance of the others. Evaluations on various
ShapeNet categories demonstrate the ability of our model to
generate editable 3D objects of improved fidelity, compared
to previous part-based generative approaches that require
3D supervision or models relying on NeRFs.


### Introduction:
1. Introduction The ability to generate high-quality 3D
shapes is a key goal in computer vision and graphics, and has been the focus of intense research over the past few years. This CVPR paper is the Open Access version, provided by the Computer Vision Foundation. Except for this watermark, it is identical to the accepted version; the final published version of the proceedings is available on IEEE Xplore. 16408 In recent years, the field of Generative Adversarial
Imitation (GAI) learns to generate high-quality 3D
images [ 40, 41,52,63,75] and videos [ 26,46,57]
through the use of generative models such as Generative Adversar-ial Imitation Learning (GAIL) [ 73,94] and implicit representations such as neural radiance fields (NeRF) [ 38,95].
In recent years, the field of Generative Adversarial Imitation
learns to generate high-

In [ ]:
batch = tokenizer(f"### INSTRUCTION\nBelow are sum cvpr2023 paper's titles, please explain main technics in these papers.\n\n### Titles:\nDong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023andDashpute_Thermal_Spread_Functions_TSF_Physics-Guided_Material_Classification_CVPR_2023andJohari_ESLAM_Efficient_Dense_SLAM_System_Based_on_Hybrid_Representation_of_CVPR_2023andGan_CNVid-3.5M_Build_Filter_and_Pre-Train_the_Large-Scale_Public_Chinese_Video-Text_CVPR_2023andChen_iQuery_Instruments_As_Queries_for_Audio-Visual_Sound_Separation_CVPR_2023\n### Explain:\n", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=200)

display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1494: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")

### INSTRUCTION
Below are sum cvpr2023 paper's titles, please explain main technic in these papers.

### Titles:
Dong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023andDashpute_Thermal_Spread_Functions_TSF_Physics-Guided_Material_Classification_CVPR_2023andJohari_ESLAM_Efficient_Dense_SLAM_System_Based_on_Hybrid_Representation_of_CVPR_2023andGan_CNVid-3.5M_Build_Filter_and_Pre-Train_the_Large-Scale_Public_Chinese_Video-Text_CVPR_2023andChen_iQuery_Instruments_As_Queries_for_Audio-Visual_Sound_Separation_CVPR_2023
### Explain:
1.Introduction Recently, deep learning has been successfully applied to various vision tasks, such as object detection, segmentation, and 3D reconstruction. Among them, monocular scene reconstruction aims at reconstructing a complete scene from only RGB images. It is a challenging task due to the high complexity of the real world, which includes physical effects ( e.g., thermal expansion, atmospheric *Corresponding author. †Corresponding author. [Color]. Material Class: Aabb Material Effects: (a) Monocular Reconstruction. (b) Deep Learning.Material Effects: (c) Fast Monocular Reconstruction. (d) Thermal Spread Functions. (e) Object Detection. (f) Segmentation. The difference is that we leverage global sparse dense grids to reconstruct the scene. The effect is shown in Figure 5. atmospheric conditions), geometric changes (e.g., changes in temperature or lighting), and physical materials. It is also highly non-trivial to associate RGB images with 3D information ( e

In [ ]:
dataset['train'][0]['title'] + "and" + dataset['train'][1]['title'] + "and" + dataset['train'][2]['title'] + "and" + dataset['train'][3]['title'] + "and" + dataset['train'][4]['title']

'Dong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023andDashpute_Thermal_Spread_Functions_TSF_Physics-Guided_Material_Classification_CVPR_2023andJohari_ESLAM_Efficient_Dense_SLAM_System_Based_on_Hybrid_Representation_of_CVPR_2023andGan_CNVid-3.5M_Build_Filter_and_Pre-Train_the_Large-Scale_Public_Chinese_Video-Text_CVPR_2023andChen_iQuery_Instruments_As_Queries_for_Audio-Visual_Sound_Separation_CVPR_2023'

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

HUGGING_FACE_USER_NAME = "bskang"
model_name = "trained_cvpr2023_data_300"

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
your_product_name_here = dataset['train'][0]['title']
your_product_description_here = dataset['train'][0]['abstract']

make_inference(your_product_name_here, your_product_description_here)

### INSTRUCTION
Below is a paper's title and abstract, please write a introduction for this paper.

### Title:
Dong_Fast_Monocular_Scene_Reconstruction_With_Global-Sparse_Local-Dense_Grids_CVPR_2023
### Abstract:
Abstract Indoor scene reconstruction from monocular images has long been sought after by augmented reality and robotics developers. Recent advances in neural field representa-tions and monocular priors have led to remarkable re-sults in scene-level surface reconstructions. The reliance on Multilayer Perceptrons (MLP), however, significantly limits speed in training and rendering. In this work, we propose to directly use signed distance function (SDF) in sparse voxel block grids for fast and accurate scene recon-struction without MLPs. Our globally sparse and locally dense data structure exploits surfaces’ spatial sparsity, en-ables cache-friendly queries, and allows direct extensions to multi-modal data such as color and semantic labels. To apply this representation to monocular scene reconstruc-tion, we develop a scale calibration algorithm for fast geo-metric initialization from monocular depth priors. We apply differentiable volume rendering from this initialization to refine details with fast convergence. We also introduce effi-cient high-dimensional Continuous Random Fields (CRFs) to further exploit the semantic-geometry consistency be-tween scene objects. Experiments show that our approach is 10× faster in training and 100× faster in rendering while achieving comparable accuracy to state-of-the-art neural implicit methods. 

### Introduction:
1. Introduction Reconstructing indoor spaces into 3D representations is a key requirement for many real-world applications, includ-ing robot navigation, immersive virtual/augmented reality experiences, and architectural design. Recent neural field representations [3, 6, 23, 37, 40] offer fast and accurate reconstruc-tions of 3D scenes with limited geometric constraints. However, these methods rely on MLPs to represent complex surfaces with weights that are not directly trans-posable to 3D points. On the other hand, there is a growing interest in developing monocular scene reconstruc-tion methods using only monocular images [9, 25, 26, 29]. These methods can be applied to unconstrained environments since they do not require geometrically annotated meshes. However, the lack of geometric constraints leads to limited re-sults for complex objects and limited application to multi-modal data, including color and semantic labels. In this work, we seek a balance between speed and accuracy in monocular scene reconstruction. Our approach is based on the fast and accurate neural volume repre-sentation with global sparse local dense grids developed by [11]. We apply this to monocular scene reconstruction by applying differentiable volume rendering with initial geome-try initialized from depth aware monocular priors. In this work, we focus on monocular scene reconstruction with the following key requirements: • Fast: we want our method to be 10× faster in training and 100× faster in re-training and evaluation. • Accuracy: our method should be comparable to or even supe-rior to neural implicit methods based on geometric annotation. • Stability: our method should be robust to noise and occlusion. • Can be extended to multi- modal data. We present a fast monocular scene recon-struction approach based on our locally dense global sparse data structure. In particular, we apply differentiable volume rendering from initial monocular depth estimates without geometric annotation. To improve speed, we develop a scale con-stantization algorithm that can estimate the volume scale based on the monocular normals and apply rendering with optimized parameters. In summary, our work is as follows: • We propose a globally sparse, locally dense data struc-ture for fast and accurate monocular scene reconstruction. • We develop a scale calibration algorithm that initial-izes monocular depth estimates and applies optimized rendering with initial scale. • We apply differentiable volume rendering from monocular depth estimates for monocular scene recon-struction with good quality and stability. • We apply continuous neural implicit methods for scene understanding with good generalization ability since our representation is high-dimensional and continuous. ### Introduction:
2. Related Work 3D Reconstruction from Monocular Images. Recent neural field representations [3, 6, 23, 37, 40] offer fast and accurate reconstructions of 3D scenes with limited geometric constraints. Global Monocular Reconstruction. Recent work focuses on monocular scene recon-struction with multi-view or RGB-D data. [25, 26] use multi-view data for semantic segmentation and 3D reconstruction. [29] uses multi-view data for semantic segmentation and object detection. [10] uses multi-view data for 3D scene understanding. [3, 11] use multi-view data for monocular scene reconstruction. [11, 19, 20] use multi-view data for monocular scene reconstruc-tion. 19] uses color and semantic labels. Our method is fast enough for real-world applications and is fast enough for training and evaluation. Semantic Monocular Reconstruction. The semantic information from monocular images enables better understanding of objects and environments. [25, 26, 29, 31, 32] use semantic labels for monocular scene reconstruc-tion. [10, 31, 32] use semantic labels for monocular object detec-tion. [10, 31] uses semantic labels for object segmentation. [19] uses semantic labels for object segmentation and segmentation. [3] uses semantic labels for geometric reconstruction. We use semantic labels for monocular scene reconstruc-tion. Semantic-based Monocular Scene Understanding. There is a strong demand for multi- modal data in the semantic and geometric understanding of indoor scenes. [22, 33] use color and semantic labels for geometric reconstruction. [2, 15] use semantic labels for object segmentation and segmentation. Our work is related to these works since we also use monocular images and semantic labels for monocular scene reconstruction. Reconstruction from Monocular Data. There are many monocular reconstruction ap-proaches. [26, 28] use depth aware monocular priors for geometric reconstruction. [20] use color and semantic labels for monocular scene understanding. [17] use semantic labels for monocular scene understanding. [18] use multi-view data for monocular scene reconstruction. Our work is fast enough for training and eval-uation since we apply differentiable volume rendering from initial monocular depth estimates. Semantic Monocular Reconstruction with Global-Sparse Local-Dense Data Structures. There are many global sparse local dense data structures proposed in the literature. [3, 11, 12, 14, 16, 18, 19, 22, 23, 25, 28, 30, 32] use voxel grids for geometric reconstruction. [25] uses voxel grids for monocular scene reconstruction. [20] uses voxel grids for object segmentation. [30] uses voxel grids for object segmentation and segmentation. [22] uses octree-based grids for geometric reconstruction. [14] uses grid-based data structures for geometric repre-sentation. [11] uses voxel blocks for monocular scene reconstruction. [12] uses grid-based data structures for semantic and semantic-geometry representations. [16] uses graph-based data structures for semantic and geometric repre-sentations. [14] uses graph-based data structures for semantic and geometric representations. 13] uses voxel blocks for monocular scene reconstruction. We use a different data structure from [11], namely, globally sparse local dense voxel grids. Our fast monocular scene reconstruction method also uses this structure. 3. Proposed Method 3.1. Fast Monocular Scene Reconstruction with Global-Sparse Local-Dense V oxel Blocks Globally Sparse. We use a globally sparse data struc-ture since it is more cache-friendly and enables fast queries. [20] uses a multi-layer perceptron (MLP)-based network for geometric repre-sentation and has fast cache-friendly queries. [25] uses a MLP-based network for monocular scene reconstruction and has fast cache-friendly queries. [18] uses a graph-based data structure for semantic and geometric repre-sentation. [14] uses a graph-based data structure for semantic repre-sentation. 13] uses voxel blocks for monocular scene reconstruction. [11] uses a MLP-based network for monocular scene reconstruction. We apply differentiable volume rendering from initial monocular depth estimates and use a globally sparse local dense voxel block grid for fast and accurate monocular scene reconstruction. Semantic Monocular Reconstruction with Global-Sparse Local-Dense V oxel Blocks. We apply differentiable volume rendering from monocular depth estimates and use a globally sparse local dense voxel block grid for monocular scene reconstruction with semantic and geometric repre-sentations. 3.2. Scale Calibration with Monocular Normal Estimates Monocular normals can be used to estimate the scale of a 3D object. [21,28] estimate the scale of a 3D object using a neural network that takes in input shape descriptors and monocular normals. [30] estimates the scale of a 3D object using a neural network that takes in input shape descriptors and camera poses. [22] estimate the scale of a 3D object using a neural network that takes in input shape descriptors and monocular normals. [15] estimates the scale of a 3D object using a neural network that takes in input shape descriptors and RGB images. [16] estimates the scale of a 3D object using a neural network that takes in input shape descriptors and RGB images. 3.3. Continual Uncertainty Rejection with High-Dimensional Continuous Random Fields High-dimensional continuous random fields (CRFs) represent continuous semantic and geometric information in an implicit form. CRFs have been used in 3D object recognition [33,34], semantic recog-nition [25] and semantic segmentation [2,15]. Uncertainty in a 3D object is represented by continuous random variables. We use a continuous CRF representation for semantic and geometric representations since they are high-dimensional and continuous. 3.4. Scale Calibration with Monocular Normal Estimates Monocular normals can be used to scale the initial geometric representation. [21] uses a neural network that takes in input shape descriptors and monocular normals to scale the initial geometric representation. [30] uses a MLP-based network that takes in input shape descriptors and monocular normals to scale the initial geometric representation. [22] uses a MLP-based network that takes in input shape descriptors and monocular normals to scale the initial geometric representation. [15] uses a graph-based representation that takes in input shape descriptors and monocular normals to scale the initial geometric representation. [16] uses a MLP-based network that takes in input shape descriptors and monocular normals to scale the initial geometric representation. 3.5. Fast Monocular Scene Reconstruction with Global-Sparse Local-Dense V oxel Blocks Our method is fast enough for real-world applications since we apply differentiable volume rendering from initial monocular depth estimates. We develop a scale calibration algorithm that initializes monocular depth estimates from monocular normals. 3.6. Fast Monocular Scene Understanding with High-Dimensional Continuous Random Fields High-dimensional continuous random fields (CRFs) represent continuous semantic and geometric information in an implicit form. CRFs have been used in 3D object recognition [33], semantic recog-nition [25] and semantic segmentation [2,15]. Uncertainty in a 3D object is represented by continuous random variables. We apply high-dimensional continuous CRFs to represent monocular scene understanding since they have good generalization ability. 3.7. Fast Monocular Scene Reconstruction with Global